# 가설 4: LLM 기반 오류 진단 → 구조 검색

## 배경

가설 3에서 확인한 사실:
- **3-1/3-2 (직접 검색)**: 오류 시그니처/교정 쌍을 검색 조건으로 사용하면 **정밀도 100%**
- **3-3 (2단계 검색)**: BM25/벡터 후보로부터 오류 패턴을 역추정하는 방식은 **노이즈에 취약**
  - 오타/철자 오류: BM25가 오타 단어를 매칭 못함 → 후보 품질 낮음
  - 복합 오류: Stage 1 후보의 오류 분포가 입력과 무관

**핵심 병목**: 입력 문장의 오류 유형을 모르는 상태에서, 말뭉치 후보로부터 역추정하는 과정

→ 이 역추정 과정을 **LLM**으로 대체하면?

## 가설

> LLM이 입력 문장의 오류를 직접 진단하고, 진단 결과를 **오류 시그니처** 및 **교정 쌍** 형식으로 변환하면,
> 가설 3의 3-1/3-2 직접 검색 파이프라인을 통해 유사 오류 사례를 정밀하게 검색할 수 있다.

### 파이프라인

```
입력 문장 → [LLM 오류 진단] → 오류 시그니처 + 교정 쌍 → [OpenSearch 직접 검색] → 유사 오류 사례
```

### 기대 효과
- LLM이 "뱡완→병원"을 직접 인식 → `CNNG:MIF` + `뱡완/NNG→병원/NNG` 생성 → 정밀 검색
- LLM이 "친구를 같이→친구와 같이"를 인식 → `FOP:REP` + `를/JKO→와/JKB` 생성 → 정밀 검색
- 3-3의 역추정 병목 없이, 직접 검색의 높은 정밀도를 활용

## 검증 계획

### Step 1. LLM 오류 진단 프롬프트 설계 및 테스트
- LLM에게 한국어 학습자 문장의 오류를 진단하도록 프롬프트 설계
- 출력 형식: 오류 위치 코드, 오류 양상, 오류 층위, 교정 형태소/품사
- 가설 3에서 사용한 4개 테스트 문장으로 진단 정확도 평가

### Step 2. 진단 결과 → 검색 쿼리 변환
- LLM 출력을 오류 시그니처(`위치:양상[:층위]`) 및 교정 쌍(`원형태소/품사→교정형태소/품사`) 형식으로 파싱
- 파싱된 결과로 가설 3의 `search_by_error_signature()` / `search_by_correction_pair()` 호출

### Step 3. 검색 결과 평가
- 테스트 문장 4개 × 검색 결과 Top-10의 적합도 수동 평가
- 가설 3의 3-3 결과와 비교: Stage 2 적합도 개선 여부 확인

---
## 구현 시작

### Step 1. LLM 오류 진단 프롬프트 설계

#### 말뭉치 오류 주석 체계

LLM이 생성해야 할 오류 시그니처/교정 쌍은 말뭉치의 주석 체계를 따라야 한다.

**오류 위치 코드** (형태소 위치):
- `CNNG`: 일반명사, `CNNP`: 고유명사, `CNP`: 대명사
- `CVV`: 동사, `CVA`: 형용사, `CVC`: 지정사, `CVX`: 보조용언
- `CXSV`: 동사파생접미사, `CXSA`: 형용사파생접미사, `CXSN`: 명사파생접미사
- `FAP`: 부사격조사, `FOP`: 목적격조사, `FNP`: 주격조사, `FGP`: 관형격조사
- `FJC`: 접속조사, `FXP`: 보조사
- `FAE`: 관형사형어미, `FED`: 연결어미, `FFE`: 종결어미
- `FPE`: 선어말어미
- `CMM`: 관형사/수사, `CMAG`: 부사

**오류 양상**:
- `REP`: 대치 (다른 형태소로 바뀌어야 함)
- `MIF`: 오형 (철자/형태 오류)
- `OM`: 누락 (있어야 할 형태소가 빠짐)
- `ADD`: 첨가 (불필요한 형태소가 추가됨)

**오류 층위** (선택):
- `PP`: 표기, `MCJ`: 형태소결합, `DS`: 방언, `ST`: 문체, `SH`: 축약/신조어

**시그니처 형식**: `위치:양상` 또는 `위치:양상:층위` (예: `CNNG:MIF`, `FAP:REP:DS`)

**교정 쌍 형식**: `원형태소/품사→교정형태소/품사` (예: `하고/JKB→와/JKB`, `뱡완/NNG→병원/NNG`)

In [ ]:
import anthropic
import json

llm = anthropic.Anthropic()

DIAGNOSIS_PROMPT = """
당신은 한국어 학습자 오류 분석 전문가입니다.
입력된 한국어 문장에서 오류를 찾아 진단해 주세요.

## 출력 형식
각 오류에 대해 다음 JSON 배열로 출력하세요. 오류가 없으면 빈 배열 []을 출력하세요.

```json
[
  {
    "error_morpheme": "오류 형태소 (원문 그대로)",
    "error_pos": "품사 태그 (NNG, JKB, EC 등 세종 태그셋)",
    "correction_morpheme": "교정 형태소",
    "correction_pos": "교정 품사 태그",
    "error_location": "오류 위치 코드 (CNNG, FAP, FOP 등)",
    "error_type": "오류 양상 (REP|MIF|OM|ADD)",
    "error_level": "오류 층위 (PP|MCJ|DS|ST|SH 또는 null)"
  }
]
```

## 오류 양상 판단 기준
- REP (대치): 문법적으로 다른 형태소를 써야 하는 경우 (조사, 어미, 어휘 선택 오류)
- MIF (오형): 철자/형태가 틀린 경우 (오타, 맞춤법 오류). 의도한 형태소는 맞지만 표기가 잘못됨
- OM (누락): 있어야 할 형태소가 빠진 경우
- ADD (첨가): 불필요한 형태소가 추가된 경우

## 오류 위치 코드
- 체언: CNNG(일반명사), CNNP(고유명사), CNP(대명사), CNNB(의존명사)
- 용언: CVV(동사), CVA(형용사), CVC(지정사), CVX(보조용언)
- 접미사: CXSV(동사파생), CXSA(형용사파생), CXSN(명사파생)
- 조사: FAP(부사격), FOP(목적격), FNP(주격), FGP(관형격), FJC(접속), FXP(보조사)
- 어미: FAE(관형사형), FED(연결), FFE(종결), FPE(선어말)
- 수식언: CMM(관형사/수사), CMAG(부사)

## 오류 층위 (해당 시에만)
- PP: 표기 오류, MCJ: 형태소 결합 오류, DS: 방언, ST: 문체, SH: 축약/신조어

## 주의사항
- 형태소 단위로 분석하세요. 어절 단위가 아닙니다.
- 품사 태그는 세종 태그셋을 따르세요 (NNG, NNP, NP, VV, VA, JKS, JKO, JKB, EC, EF, EP 등).
- 오류가 여러 개이면 모두 나열하세요.
- JSON만 출력하세요. 설명은 불필요합니다.
"""

def diagnose_errors(sentence: str) -> list[dict]:
    """LLM에게 한국어 학습자 문장의 오류를 진단 요청"""
    response = llm.messages.create(
        model="claude-sonnet-4-5-20250929",
        max_tokens=1024,
        messages=[
            {"role": "user", "content": f"{DIAGNOSIS_PROMPT}\n\n입력 문장: {sentence}"}
        ],
    )
    text = response.content[0].text.strip()
    # JSON 블록 추출 (```json ... ``` 또는 순수 JSON)
    if "```json" in text:
        text = text.split("```json")[1].split("```")[0].strip()
    elif "```" in text:
        text = text.split("```")[1].split("```")[0].strip()
    return json.loads(text)

print("LLM 진단 함수 정의 완료")

In [ ]:
def format_diagnosis(errors: list[dict]) -> dict:
    """LLM 진단 결과를 오류 시그니처 및 교정 쌍으로 변환"""
    signatures = []
    pairs = []
    for e in errors:
        # 시그니처: 위치:양상[:층위]
        sig = f"{e['error_location']}:{e['error_type']}"
        if e.get('error_level'):
            sig += f":{e['error_level']}"
        signatures.append(sig)
        # 교정 쌍: 원형태소/품사→교정형태소/품사
        if e['error_type'] == 'OM':
            pair = f"∅→{e['correction_morpheme']}/{e['correction_pos']}"
        elif e['error_type'] == 'ADD':
            pair = f"{e['error_morpheme']}/{e['error_pos']}→ADD/{e['error_pos']}"
        else:
            pair = f"{e['error_morpheme']}/{e['error_pos']}→{e['correction_morpheme']}/{e['correction_pos']}"
        pairs.append(pair)
    return {"signatures": signatures, "correction_pairs": pairs}

print("변환 함수 정의 완료")

#### 테스트: 가설 3에서 사용한 4개 문장으로 LLM 진단 정확도 평가

In [ ]:
test_sentences = [
    {
        "sentence": "저는 어제 뱡완에 갔어요.",
        "description": "오타 오류 (뱡완 → 병원)",
        "expected_sig": "CNNG:MIF",
        "expected_pair": "뱡완/NNG→병원/NNG",
    },
    {
        "sentence": "저는 친구를 같이 영화를 봤어요.",
        "description": "조사 대치 (를 → 와)",
        "expected_sig": "FOP:REP",
        "expected_pair": "를/JKO→와/JKB",
    },
    {
        "sentence": "공부가 힘든입니다.",
        "description": "어미 오류 (힘든입니다 → 힘듭니다)",
        "expected_sig": "FAE:REP",
        "expected_pair": "ㄴ/ETM→ㅂ/EP",
    },
    {
        "sentence": "내 월꺼 아네 있는데요",
        "description": "복합 오류 (월꺼 → 윗것, 아네 → 안에)",
        "expected_sig": ["CNNG:MIF", "CNNG:MIF"],
        "expected_pair": ["월꺼/NNG→윗것/NNG", "아네/NNG→안에/NNG"],
    },
]

results = []
for t in test_sentences:
    print("=" * 70)
    print(f"입력: {t['sentence']}")
    print(f"설명: {t['description']}")
    print(f"기대 시그니처: {t['expected_sig']}")
    print(f"기대 교정 쌍: {t['expected_pair']}")
    print()

    errors = diagnose_errors(t['sentence'])
    formatted = format_diagnosis(errors)

    print(f"LLM 진단 결과:")
    for e in errors:
        print(f"  - {e['error_morpheme']}({e['error_pos']}) → {e['correction_morpheme']}({e['correction_pos']})")
        print(f"    위치: {e['error_location']}, 양상: {e['error_type']}, 층위: {e.get('error_level', '-')}")
    print()
    print(f"생성 시그니처: {formatted['signatures']}")
    print(f"생성 교정 쌍: {formatted['correction_pairs']}")
    print()

    results.append({
        "sentence": t['sentence'],
        "expected_sig": t['expected_sig'],
        "generated_sig": formatted['signatures'],
        "expected_pair": t['expected_pair'],
        "generated_pair": formatted['correction_pairs'],
        "raw_errors": errors,
    })

### Step 2. 진단 결과로 OpenSearch 검색

In [ ]:
from opensearchpy import OpenSearch

client = OpenSearch(
    hosts=[{'host': '172.30.1.81', 'port': 9200}],
    http_auth=None,
    use_ssl=False,
    verify_certs=False,
)

INDEX_NAME = 'korean_test'

def search_by_error_signature(signatures, size=10):
    """오류 시그니처로 동일 패턴의 문장을 검색"""
    query = {
        "size": size,
        "query": {"terms": {"error_signatures": signatures}},
        "_source": ["original_text", "error_signatures", "correction_pairs"],
    }
    resp = client.search(index=INDEX_NAME, body=query)
    return resp['hits']['hits']

def search_by_correction_pair(pairs, size=10):
    """교정 쌍으로 동일 오류의 문장을 검색"""
    query = {
        "size": size,
        "query": {"terms": {"correction_pairs": pairs}},
        "_source": ["original_text", "error_signatures", "correction_pairs"],
    }
    resp = client.search(index=INDEX_NAME, body=query)
    return resp['hits']['hits']

print("OpenSearch 클라이언트 및 검색 함수 준비 완료")

In [ ]:
# Step 2 테스트: LLM 진단 결과로 직접 검색
for r in results:
    print("=" * 70)
    print(f"입력: {r['sentence']}")
    print(f"생성 시그니처: {r['generated_sig']}")
    print(f"생성 교정 쌍: {r['generated_pair']}")
    print()

    # 시그니처 기반 검색
    if r['generated_sig']:
        print(f"--- 시그니처 검색 결과 (Top 5) ---")
        hits = search_by_error_signature(r['generated_sig'], size=5)
        for h in hits:
            src = h['_source']
            print(f"  [{h['_id']}] {src['original_text']}")
            print(f"    시그니처: {src['error_signatures']}")
            print(f"    교정 쌍:  {src['correction_pairs']}")
        print()

    # 교정 쌍 기반 검색
    if r['generated_pair']:
        print(f"--- 교정 쌍 검색 결과 (Top 5) ---")
        hits = search_by_correction_pair(r['generated_pair'], size=5)
        for h in hits:
            src = h['_source']
            print(f"  [{h['_id']}] {src['original_text']}")
            print(f"    시그니처: {src['error_signatures']}")
            print(f"    교정 쌍:  {src['correction_pairs']}")
    print()

### Step 3. 결과 평가

가설 3의 3-3 결과와 비교하여:
1. LLM이 생성한 시그니처/교정 쌍이 말뭉치 주석 체계와 얼마나 일치하는가?
2. 생성된 검색 조건으로 검색한 결과가 입력 문장의 오류 유형과 얼마나 관련 있는가?
3. 가설 3 3-3의 역추정 대비 검색 정밀도가 개선되었는가?